<a href="https://colab.research.google.com/github/XuyangAbert/ALCS/blob/main/deepALplus_alcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/XuyangAbert/DeepALCS

Cloning into 'DeepALCS'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Receiving objects: 100% (81/81), 76.72 KiB | 19.18 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [1]:
import sys
sys.path.append("/content/DeepALCS/deepALplus-master")

In [2]:
!pip install numpy
!pip install scipy
!pip install torch
!pip install torchvision
!pip install scikit-learn
!pip install tqdm
!pip install ipdb==0.13.9
!pip install openml==0.12.2
!pip install faiss-gpu==1.7.2
!pip install toma==1.1.0
!pip install opencv-python==4.5.5.64
!pip install wilds==2.0.0

In [3]:
import argparse
import numpy as np
import warnings
import torch
from utils import get_dataset, get_net, get_net_lpl, get_net_waal, get_strategy
from pprint import pprint

torch.set_printoptions(profile='full')

import sys
import os
import re
import random
import math
import datetime

# import arguments
from parameters import *
from utils import *

# # parameters
# args_input = arguments.get_args()
# NUM_QUERY = args_input.batch
# NUM_INIT_LB = args_input.initseed
# NUM_ROUND = int(args_input.quota / args_input.batch)
# DATA_NAME = args_input.dataset_name
# STRATEGY_NAME = args_input.ALstrategy


# SEED = args_input.seed
# os.environ['TORCH_HOME']='./basicmodel'
# os.environ["CUDA_VISIBLE_DEVICES"] = str(args_input.gpu)

# # fix random seed
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.backends.cudnn.enabled  = True
# torch.backends.cudnn.benchmark= True

In [4]:
initseed = 100
ALstrategy = "ALCS_Diversity"
NUM_QUERY = 100
NUM_ROUND = 3
DATA_NAME = "CIFAR10"

In [ ]:
# device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# #recording
# sys.stdout = Logger(os.path.abspath('') + '/logfile/' + DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input.quota) + '_normal_log.txt')
warnings.filterwarnings('ignore')

# start experiment

iteration = 5

all_acc = []
acq_time = []

args_input = {}
dataset_name = DATA_NAME

# repeate # iteration trials
while (iteration > 0):
	iteration = iteration - 1
	# data, network, strategy
	args_task = args_pool[dataset_name]

	dataset = get_dataset(dataset_name, args_task)				# load dataset
	if ALstrategy == 'LossPredictionLoss':
		net = get_net_lpl(dataset_name, args_task, device)		# load network
	elif ALstrategy == 'WAAL':
		net = get_net_waal(dataset_name, args_task, device)		# load network
	else:
		net = get_net(dataset_name, args_task, device)			# load network
	strategy = get_strategy(ALstrategy, dataset, net, args_input, args_task)  # load strategy
	print(strategy)
	start = datetime.datetime.now()
  # generate initial labeled pool
	dataset.initialize_labels(initseed)
  #record acc performance
	acc = np.zeros(NUM_ROUND + 1)
  # only for special cases that need additional data
	new_X = torch.empty(0)
	new_Y = torch.empty(0)
	# print info
	print(args_task)
	print('RANDOM SEED {}'.format(4666))
	print(type(strategy).__name__)
	# One-time query for ALCS-Diversity
	if ALstrategy == 'ALCS_Diversity':
		strategy.train()
		q_idxs = strategy.query(NUM_QUERY*(NUM_ROUND))
		strategy.update(q_idxs)
		strategy.train()
		preds = strategy.predict(dataset.get_test_data())
		acc = dataset.cal_test_acc(preds)
		print('testing accuracy {}'.format(acc))
	else:
		# round 0 accuracy
		if ALstrategy == 'WAAL':
			strategy.train(model_name = ALstrategy)
		else:
			strategy.train()
		preds = strategy.predict(dataset.get_test_data())
		acc[0] = dataset.cal_test_acc(preds)
		print('Round 0\ntesting accuracy {}'.format(acc[0]))
		print('\n')

		# round 1 to rd
		for rd in range(1, NUM_ROUND+1):
			# print('Round {}'.format(rd))
			high_confident_idx = []
			high_confident_pseudo_label = []
			# query
			if 'CEALSampling' in ALstrategy:
				q_idxs, new_data = strategy.query(NUM_QUERY, rd, option = ALstrategy[13:])
			else:
				q_idxs = strategy.query(NUM_QUERY)

			# update
			strategy.update(q_idxs)

			#train
			if 'CEALSampling' in ALstrategy:
				strategy.train(new_data)
			elif ALstrategy == 'WAAL':
				strategy.train(model_name = ALstrategy)
			else:
				strategy.train()

			# round rd accuracy
			preds = strategy.predict(dataset.get_test_data())
			acc[rd] = dataset.cal_test_acc(preds)
			# print('testing accuracy {}'.format(acc[rd]))
			# print('\n')
		# print results
		# print('SEED {}'.format(SEED))
		print(type(strategy).__name__)
		print(acc[-1])
		all_acc.append(acc[-1])

	# #save model
	# timestamp = re.sub('\.[0-9]*','_',str(datetime.datetime.now())).replace(" ", "_").replace("-", "").replace(":","")
	# os.makedirs('./modelpara/', exist_ok=True)
	# model_path = './modelpara/'+timestamp + DATA_NAME+ '_'  + ALstrategy + '_' + str(NUM_QUERY) + '_' + str(initseed) +  '_' + str(1000)  +'.params'
	# end = datetime.datetime.now()
	# acq_time.append(round(float((end-start).seconds),3))
	# torch.save(strategy.get_model().state_dict(), model_path)

# # cal mean & standard deviation
# acc_m = []
# file_name_res_tot = DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input.quota) + '_normal_res_tot.txt'
# file_res_tot =  open(os.path.join(os.path.abspath('') + '/results', '%s' % file_name_res_tot),'w')

# file_res_tot.writelines('dataset: {}'.format(DATA_NAME) + '\n')
# file_res_tot.writelines('AL strategy: {}'.format(STRATEGY_NAME) + '\n')
# file_res_tot.writelines('number of labeled pool: {}'.format(NUM_INIT_LB) + '\n')
# file_res_tot.writelines('number of unlabeled pool: {}'.format(dataset.n_pool - NUM_INIT_LB) + '\n')
# file_res_tot.writelines('number of testing pool: {}'.format(dataset.n_test) + '\n')
# file_res_tot.writelines('batch size: {}'.format(NUM_QUERY) + '\n')
# file_res_tot.writelines('quota: {}'.format(NUM_ROUND*NUM_QUERY)+ '\n')
# file_res_tot.writelines('time of repeat experiments: {}'.format(args_input.iteration)+ '\n')


Files already downloaded and verified
Files already downloaded and verified
{'n_epoch': 30, 'name': 'CIFAR10', 'transform_train': Compose(
    RandomCrop(size=(32, 32), padding=4)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616))
), 'transform': Compose(
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616))
), 'loader_tr_args': {'batch_size': 128, 'num_workers': 4}, 'loader_te_args': {'batch_size': 1000, 'num_workers': 4}, 'num_class': 10, 'pretrained': False, 'optimizer': 'Adam', 'optimizer_args': {'lr': 0.001}}
RANDOM SEED 4666
ALCS_Diversity


100%|███████████████████████████████████████████████████████████████| 30/30 [00:06<00:00,  4.76it/s]


fps-clustering stage finish!


100%|███████████████████████████████████████████████████████████████| 30/30 [00:08<00:00,  3.73it/s]


testing accuracy 0.3607
Files already downloaded and verified
Files already downloaded and verified
{'n_epoch': 30, 'name': 'CIFAR10', 'transform_train': Compose(
    RandomCrop(size=(32, 32), padding=4)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616))
), 'transform': Compose(
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616))
), 'loader_tr_args': {'batch_size': 128, 'num_workers': 4}, 'loader_te_args': {'batch_size': 1000, 'num_workers': 4}, 'num_class': 10, 'pretrained': False, 'optimizer': 'Adam', 'optimizer_args': {'lr': 0.001}}
RANDOM SEED 4666
ALCS_Diversity


100%|███████████████████████████████████████████████████████████████| 30/30 [00:06<00:00,  4.67it/s]
